# Cliente de la rama BATCH

Este cliente se limita a leer los datos del topic y los vuelca al fichero historic.csv. Podríamos tener varios procesos iguales para permitir alta disponibilidad y mejor rendimieento, pero:
- deberiamos asegurarnos de que varios procesos pueden escribir a la vez en el fichero,
- incluso aunque el topic esté formado por varias particiones para facilitar el trabajo de la rama real-time, este cliente tan ligero que probablemente añadir varios no mejore el rendimiento.

Si el destino fuera una base de datos (lo más habitual en el mundo real) probablemente podríamos obviar la primera consideración.

In [1]:
import pandas as pd
from ejercicios.houses import BATCH_GROUP, HISTORIC, TOPIC

ds = pd.read_csv(HISTORIC)

In [4]:
from confluent_kafka import Consumer, KafkaError
import json

c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': BATCH_GROUP,
    'auto.offset.reset': 'earliest'
})

c.subscribe([TOPIC])

while True:
    msg = c.poll(1.0)

    if msg is None:
        continue
    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    data_json = msg.value().decode('utf-8')
    data_dict = json.loads(data_json)

    print('Received message: {}:{}'.format(data_dict.get('price'), data_dict.get('yr_built')))

    ds = ds.append(data_dict, ignore_index=True)
    ds.to_csv(HISTORIC)

c.close()


Received message: 350000.0:1948
Received message: 269000.0:1986
Received message: 739000.0:1981
Received message: 452000.0:1946
Received message: 330000.0:2007
Received message: 655000.0:1983
Received message: 418500.0:2007
Received message: 731500.0:2013
Received message: 557500.0:1959
Received message: 175000.0:1954
Received message: 825000.0:1938
Received message: 380000.0:2011
Received message: 143000.0:1969
Received message: 542000.0:1955
Received message: 324000.0:1999
Received message: 404763.0:2014
Received message: 950000.0:1986
Received message: 599000.0:1920
Received message: 375000.0:1937
Received message: 250000.0:1960
Received message: 227000.0:1944
Received message: 625000.0:1951
Received message: 530000.0:2012
Received message: 750000.0:1915
Received message: 505000.0:2005
Received message: 249900.0:1994
Received message: 659500.0:1957
Received message: 899000.0:1949
Received message: 575000.0:1974
Received message: 654000.0:1997
Received message: 340000.0:1951
Received

KeyboardInterrupt: 